In [46]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.datasets import load_boston

In [2]:
boston = load_boston()

In [3]:
data = pd.DataFrame(boston.data, columns=boston.feature_names)
data['HomeVal50'] = boston.target

In [6]:
data.head(15)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,HomeVal50
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,1
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,1
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,1
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,1
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,1
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,1
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,1
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,0
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,0


In [5]:
def to_label(data, target, percentile):
    '''
    Input: data, name of target column, percentile to partition data
    Output: data, but with the target column values
    changed from continuous to categorial (classes)
    '''
    frac = percentile / 100.0
    part_val = data[target].quantile(frac)
    data[target] = [1 if d > part_val else 0 for d in data[target]]
    return data

label = 'HomeVal50'
data = to_label(data, label, 50)

In [7]:
sample = data.head(30)

In [8]:
sample.shape[0]

30

In [47]:
# Split the data into k folds
def cross_val_split(data, folds, label, index):
    data_idx = []
    indices = [i for i in range(data.shape[0])]
    
    fold_size = int(len(data)/folds)
    for i in range(folds):
        fold_idx = []
        while len(fold_idx) < fold_size:
            idx = random.randrange(len(indices))
            fold_idx.append(indices.pop(idx))
        data_idx.append(fold_idx)
    
    test_idx = data_idx[index]
    del data_idx[index]
    train_idx = [item for sublist in data_idx for item in sublist]
    
    test = data.iloc[test_idx]
    train = data.iloc[train_idx]
    
    return train, test

In [52]:
train, test = cross_val_split(sample, 5, 'HomeVal50', 0)

In [53]:
train.shape

(24, 14)

In [54]:
test.shape

(6, 14)

In [72]:
def calculate_parameters(data):
    label = 'HomeVal50'
    num_dims = 1
    
    # get the groups
    grouped = data.groupby(data.loc[:,label])
    classes = [k for k in grouped.groups.keys()]

    # mean for each class:
    data_class = {}
    X = {}
    means = {}
    for k in classes:
        data_class[k] = grouped.get_group(k)
        X[k] = data_class[k].drop(label,axis=1)
        means[k] = np.array(np.mean(X[k]))

    # mean of the total data
    mean_total = np.array(np.mean(data.drop(label,axis=1)))

    # between covariance matrix
    S_B = np.zeros((X[0].shape[1], X[0].shape[1]))
    for k in X.keys():
        S_B += np.multiply(len(X[k]),
                            np.outer((means[k] - mean_total),
                            np.transpose(means[k] - mean_total)))

    # within covariance matrix
    S_W = np.zeros((X[0].shape[1], X[0].shape[1]))
    for k in classes:
        S_k = X[k] - means[k]
        S_W += np.dot(S_k.T, S_k)

    # eigendecomposition
    S = np.dot(np.linalg.pinv(S_W), S_B)
    eigval, eigvec = np.linalg.eig(S)

    # sort eigenvalues in decreasing order
    eig = [(eigval[i], eigvec[:,i]) for i in range(len(eigval))]
    eig = sorted(eig, key=lambda x: x[0], reverse=True)

    # only take the top (number of dimensions projected into) vectors
    w = np.array([eig[i][1] for i in range(num_dims)])
    
    return w

In [73]:
calculate_parameters(data)

array([[  6.96521067e-04+0.j,  -3.67346873e-04+0.j,  -3.02407772e-03+0.j,
         -1.41998627e-01+0.j,   9.81980384e-01+0.j,  -7.80610918e-02+0.j,
          3.86049982e-03+0.j,   6.78726687e-02+0.j,  -1.94971166e-02+0.j,
          8.24356172e-04+0.j,   6.12543321e-02+0.j,  -4.20386807e-04+0.j,
          2.63235097e-02+0.j]])

### Gaussian modeling

In [71]:
priors = g_means = g_cov = {}
for k in X.keys():
    priors[k] = X[k].shape[0] / sample.shape[0]
    g_means[k] = np.mean(X[k])
    g_cov[k] = np.cov(X[k],rowvar=False)
print(g_cov[0].shape)

(13, 13)
